In [ ]:
pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
# 08_callbacks_tensorboard_keras_tuner.ipynb

# 📌 Objective: Integrate Keras Tuner + TensorBoard + Callbacks for model optimization

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import keras_tuner as kt
import datetime, os

# ✅ Load and preprocess data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# ✅ Model builder for Keras Tuner
def model_builder(hp):
    hp_units = hp.Int('units', min_value=64, max_value=256, step=64)
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)

    model = keras.Sequential([
        layers.Dense(hp_units, activation='relu', input_shape=(784,)),
        layers.Dropout(hp_dropout),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ✅ TensorBoard logging
log_dir = os.path.join("logs", "tuner", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = callbacks.TensorBoard(log_dir=log_dir)

# ✅ EarlyStopping
early_stop_cb = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# ✅ Keras Tuner (RandomSearch)
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='keras_tuner_logs',
    project_name='mnist_tuning'
)

# ✅ Search best hyperparameters
tuner.search(x_train, y_train,
             epochs=15,
             validation_split=0.2,
             callbacks=[tensorboard_cb, early_stop_cb],
             verbose=1)

# ✅ Retrieve best model
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"\nBest model test accuracy: {test_acc:.4f}")

# 📊 View results in TensorBoard
# In Colab, uncomment:
# %tensorboard --logdir logs/tuner

Trial 5 Complete [00h 03m 13s]
val_accuracy: 0.9788749814033508

Best val_accuracy So Far: 0.9790416657924652
Total elapsed time: 00h 15m 25s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9764 - loss: 0.0869

Best model test accuracy: 0.9796
